# Yapay Sinir Ağları (Artifical Neural Network)

İnsan beyninin bilgi işleme şeklini referans alan sınflandırma ve regresyon problemleri için kullanılabilen kuvvetli makine öğrendmesi algoritmalarından birisidir.


Amaç en küçük hata ile tahmin yapabilecek katsayılara erişmektir.

In [2]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [3]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
#Veri setindeki kategorik değişkenler için dummy değişkenler oluşturduk
#One hot encoding yöntemi ile.
y = df["Salary"] #Bağımlı değişkenimiz
X_ = df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
#Veri setimizdeki bağımlı değişken ve kategorik değişkenlerimizin orjinal hallerini uçurduk
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
#Bu kategorik değişkenlerin dummy versyonlarını ve diğer bağımsız
#değişken değerlerimizi bir arada tutarak bağımsız değişkenlerimizi
#X olarak oluşturmuş olduk
X_train,X_test,y_train,y_test  =train_test_split(X,y,test_size = 0.25,random_state=42)

### Model ve Tahmin

In [4]:
# Değişkenlerimize bir standartlaştırma işlemi gerçekleştiricez
# Mokine algoritmalarının çoğu bunu sever fakat bazı
#algoritmalar heterojen veri setinde iyi çalışırken
#bazıları homojen veri setlerinde iyi çalışır
#Yapay sinir ağı homojen veri setleri üzerinde daha iyi 
#çalışan  bir algoritmadır.
#Yapılan türevleme işlemlerinde  ortaya çıkmış olan
#aykırıkların ciddi problemlere yol açması söz konusu
#Bundan dolayı yapay sinir ağlarını standartlaştırma 
#sonrası kullanmak daha sağlıklı olmaktadır.

In [5]:
scaler = StandardScaler()

In [6]:
scaler.fit(X_train)#train setimizi standatlaştırma işlemi
X_train_scaled = scaler.transform(X_train) #x traini ölçeklendirmek

In [7]:
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test) #aynısını test seti için yaptık

In [8]:
#Çok katmanlı algılayıcı model: hiç paremetre girmedik.
mlp_model = MLPRegressor().fit(X_train_scaled,y_train)

D:\Anakonda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [9]:
mlp_model #birçok paremetre ön tanımlı olarak kullanılmış

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [10]:
mlp_model.predict(X_test_scaled)[0:5] #5 değer tahmini

array([18.47782197, 31.04231092, 62.56362059, 13.14560671, 24.50412375])

In [11]:
y_pred = mlp_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

661.1950507257158

## Model Tuning

In [13]:
mlp_params = {"alpha":[0.1,0.01,0.02,0.001,0.0001], #lambdalar
             "hidden_layer_sizes":[(10,20),(5,5),(100,100)]} #katman ve hücre oluşacak girdiğimiz değerlere göre iki değer girdiğimizde iki tane gizli katman koy diyoruz

In [19]:
mlp_cv_model = GridSearchCV(mlp_model,mlp_params,cv = 10,verbose = 2,n_jobs = -1).fit(X_train_scaled,y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    6.1s finished
D:\Anakonda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [20]:
mlp_cv_model.best_params_

{'alpha': 0.1, 'hidden_layer_sizes': (100, 100)}

In [21]:
#Final
mlp_tuned = MLPRegressor(alpha = 0.01,hidden_layer_sizes = (100,200)).fit(X_train_scaled,y_train)

D:\Anakonda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [22]:
y_pred = mlp_tuned.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

350.59564160382143